<a href="https://colab.research.google.com/github/Anmollllll/SuicideTextDetection_and_ResponseModel/blob/main/ResponseBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install  langchain-google-genai
!pip install langchain-core
!pip install pyngrok

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os
from datetime import datetime

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# page setup
st.set_page_config(page_title="Crisis Support Chatbot", page_icon="💬", layout="wide")
st.title("🧠 Suicide Risk Detection & Support Chatbot")


BERT_MODEL_PATH = "/content/drive/MyDrive/SusDetect/results/checkpoint-4269"
os.environ["GOOGLE_API_KEY"] = "...........your_api_code........."
Chatmodel = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# system_prompt
SYSTEM_PROMPT = """
You are a compassionate AI crisis support companion designed to provide immediate emotional support to individuals experiencing suicidal thoughts or mental health crises.

Your Core Responsibilities:

1. Soothe and Stabilize
- Use gentle, calming language to help reduce immediate distress
- Suggest grounding techniques like deep breathing or mindfulness when appropriate
- Help the person feel heard and less alone in their struggle

2. Build Trust Through Empathy
- Be non-judgmental, warm, and accepting of all feelings
- Validate their pain without minimizing it
- Listen actively and reflect back what you hear to show understanding
- Maintain confidentiality in our conversation

3. Understand the Crisis
- Ask open-ended questions to gently explore what's happening
- Identify specific triggers, stressors, or situations contributing to their pain
- Acknowledge the complexity of their experience

4. Collaborate on Coping Strategies
- Help them identify their existing strengths and past coping mechanisms
- Explore small, manageable steps they can take right now
- Encourage problem-solving without forcing solutions
- Discuss people or resources they might reach out to

5. Encourage Safety and Professional Support
- Gently encourage them to commit to staying safe
- Remind them that professional help is available and valuable
- Provide crisis helpline information when appropriate


Crisis Resources to Share When Needed:
Direct the users to one of these website links or based on which country they are currently on,find the emergency numbers from these websites and help them.
https://heartfeltpresence.org/global-suicide-hotlines/?utm_source=chatgpt.com "Global Emergency Hotlines - Heartfelt Presence"
https://trystressmanagement.com/mental-health-resources/asias-hotlines/?utm_source=chatgpt.com "Asia's Crisis Centres and Hotlines - Try Stress Management"
https://www.positivminds.com/te/general-5?utm_source=chatgpt.com "Help | PositivMinds"


Tone and Approach:
- Be warm, genuine, and human in your responses
- Avoid clinical jargon or overly formal language
- Show care through your words, not just procedures
- Balance hope with realism - don't offer toxic positivity
- Meet them where they are emotionally

MOST IMPORTANT NOTE:
Always provide positive response to the user input and be positive to the user no matter how the critical situtation is.
"""

#session state init
if "post_input" not in st.session_state:
    st.session_state.post_input = ""
if "chat_active" not in st.session_state:
    st.session_state.chat_active = False
if "current_chat_messages" not in st.session_state:
    st.session_state.current_chat_messages = []
if "active_post_text" not in st.session_state: # Stores the original post that started the current chat
    st.session_state.active_post_text = ""
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []  # list of dicts: {id, post, chat_messages, ts}
if "selected_history_id" not in st.session_state:
    st.session_state.selected_history_id = None
if "chat_input_key" not in st.session_state: # Used to reset the chat text_input
    st.session_state.chat_input_key = 0
if "clear_post_input_flag" not in st.session_state: # NEW: Flag to control post_input clearing
    st.session_state.clear_post_input_flag = False
if "last_non_suicidal_prediction" not in st.session_state:
    st.session_state.last_non_suicidal_prediction = None # to store {label, prob}

# This ensures st.session_state.post_input is only modified BEFORE the widget is created.
if st.session_state.clear_post_input_flag:
    st.session_state.post_input = ""
    st.session_state.clear_post_input_flag = False

# loading bert model
@st.cache_resource
def load_bert_model(path):
    tokenizer = AutoTokenizer.from_pretrained(path)
    model = AutoModelForSequenceClassification.from_pretrained(path)
    return tokenizer, model

tokenizer, bert_model = load_bert_model(BERT_MODEL_PATH)

def predict_suicidal(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = bert_model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    label_idx = torch.argmax(probs).item()
    suicidal_prob = float(probs[0][1].item())
    label = "Suicidal" if label_idx == 1 else "Non-Suicidal"
    return label, suicidal_prob

#chat interaction functions

def start_new_chat_session(post_text):
    """Initializes a new chat session with the bot based on the user's post."""
    st.session_state.current_chat_messages = [
        SystemMessage(content=SYSTEM_PROMPT),
        HumanMessage(content=post_text)
    ]
    with st.spinner("Connecting to bot..."):
        bot_response = Chatmodel.invoke(st.session_state.current_chat_messages)
    st.session_state.current_chat_messages.append(AIMessage(content=bot_response.content))
    st.session_state.chat_active = True
    st.session_state.active_post_text = post_text
    st.session_state.chat_input_key += 1
    st.rerun()

def send_chat_message(user_message):
    """Sends a user message and gets a bot response in an active chat."""
    if not user_message.strip():
        return

    st.session_state.current_chat_messages.append(HumanMessage(content=user_message))
    with st.spinner("Bot is typing..."):
        bot_response = Chatmodel.invoke(st.session_state.current_chat_messages)
    st.session_state.current_chat_messages.append(AIMessage(content=bot_response.content))
    st.session_state.chat_input_key += 1
    st.rerun()

def end_chat_session_logic(trigger_clear_post_input=False):
    """
    Handles the logic of ending a chat and saving to history.
    Sets a flag if post_input should be cleared on the next rerun.
    """
    if st.session_state.chat_active and st.session_state.current_chat_messages:
        chat_for_history = [msg for msg in st.session_state.current_chat_messages if not isinstance(msg, SystemMessage)]

        entry = {
            "id": len(st.session_state.chat_history) + 1,
            "post": st.session_state.active_post_text,
            "chat_messages": chat_for_history,
            "ts": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        st.session_state.chat_history.insert(0, entry)

    # Reset active chat state
    st.session_state.chat_active = False
    st.session_state.current_chat_messages = []
    st.session_state.active_post_text = ""
    st.session_state.selected_history_id = None
    st.session_state.chat_input_key += 1

    if trigger_clear_post_input:
        st.session_state.clear_post_input_flag = True

def handle_exit_chat_button_click():
    """Callback for the Exit Chat button."""
    end_chat_session_logic(trigger_clear_post_input=True)
    st.success("Chat ended and saved to history.")
    st.rerun()

def load_chat_from_history(history_item):
    """Loads a selected chat from history to view or continue."""
    st.session_state.current_chat_messages = [SystemMessage(content=SYSTEM_PROMPT)] + history_item["chat_messages"]
    st.session_state.active_post_text = history_item["post"]
    st.session_state.chat_active = True
    st.session_state.selected_history_id = history_item["id"]
    st.session_state.post_input = history_item["post"] # Pre-fill post input if continuing
    st.session_state.chat_input_key += 1
    st.rerun()

def delete_chat_from_history(history_id):
    """Deletes a chat entry from the history."""
    was_active_chat_deleted = (st.session_state.selected_history_id == history_id) or \
                             (st.session_state.chat_active and any(item["id"] == history_id for item in st.session_state.chat_history if item["post"] == st.session_state.active_post_text))

    st.session_state.chat_history = [
        item for item in st.session_state.chat_history if item["id"] != history_id
    ]
    if was_active_chat_deleted:
        end_chat_session_logic(trigger_clear_post_input=True) # End current chat and clear post_input
    else:
        pass
    st.rerun()

# UI Layout

# Sidebar for Chat History
with st.sidebar:
    st.header("🗂️ Chat History")
    if st.session_state.chat_history:
        for i, chat_entry in enumerate(st.session_state.chat_history):
            is_current_active_chat = (st.session_state.chat_active and st.session_state.active_post_text == chat_entry["post"] and
                                      [msg for msg in st.session_state.current_chat_messages if not isinstance(msg, SystemMessage)] == chat_entry["chat_messages"])
            is_selected_in_history = (st.session_state.selected_history_id == chat_entry["id"])

            with st.expander(
                f"**{chat_entry['post'][:45]}...** (ID: {chat_entry['id']}) - {chat_entry['ts']}",
                expanded=is_current_active_chat or is_selected_in_history
            ):
                st.markdown(f"**Original Post:** {chat_entry['post']}")
                for msg in chat_entry["chat_messages"]:
                    if isinstance(msg, HumanMessage):
                        st.markdown(f"**You:** {msg.content}")
                    elif isinstance(msg, AIMessage):
                        st.markdown(f"**Bot:** {msg.content}")

                col1, col2 = st.columns([1,1])
                with col1:
                    if st.button("Continue Chat", key=f"continue_{chat_entry['id']}"):
                        load_chat_from_history(chat_entry)
                with col2:
                    if st.button("Delete", key=f"delete_history_{chat_entry['id']}"):
                        delete_chat_from_history(chat_entry["id"])
    else:
        st.info("No saved chats yet.")

# Main content area
st.subheader("💬 Enter a post or text to analyze")

st.text_area(
    "Post / status / text",
    value=st.session_state.post_input,
    height=150,
    key="post_input"
)

if st.button("🔍 Analyze Post", key="analyze_button"):
    if not st.session_state.post_input.strip():
        st.warning("Please enter some text to analyze.")
    else:
        label, suicidal_prob = predict_suicidal(st.session_state.post_input.strip())

        st.markdown(f"### 🧠 Prediction: **{label}**")
        st.markdown(f"**Suicidal probability:** {suicidal_prob*100:.2f}%")
        st.progress(suicidal_prob)

        if label == "Suicidal":
            st.error("⚠️ Suicidal content detected — initiating support chat.")
            start_new_chat_session(st.session_state.post_input.strip())
        else:
            st.success("✅ Non-suicidal detected — no chat started.")
            st.session_state.last_non_suicidal_prediction = {
                "label":label,
                "prob":suicidal_prob
            }

# Active Chatbot Interface (appears only when chat_active is True)
if st.session_state.chat_active:
    st.markdown("---")
    st.markdown("## 🤖 Crisis Support Chatbot")
    st.info(f"Chat initiated for post: **{st.session_state.active_post_text[:100]}...**")

    chat_display_messages = [
        msg for msg in st.session_state.current_chat_messages if not isinstance(msg, SystemMessage)
    ]
    for msg in chat_display_messages:
        if isinstance(msg, HumanMessage):
            with st.chat_message("user"):
                st.markdown(msg.content)
        elif isinstance(msg, AIMessage):
            with st.chat_message("assistant"):
                st.markdown(msg.content)

    # Move st.chat_input OUTSIDE the st.form
    user_message = st.chat_input("Your message:", key=f"chat_input_{st.session_state.chat_input_key}")

    if user_message:
        send_chat_message(user_message)

    with st.form("exit_chat_form", clear_on_submit=False):
        if st.form_submit_button("❌ Exit Chat"):
            handle_exit_chat_button_click()


In [ ]:
!ngrok authtoken #your ngrok api key

In [ ]:
from pyngrok import ngrok


ngrok.kill()

# Start new tunnel on port 8501
public_url = ngrok.connect(addr="8501", proto="http")
print("Streamlit App URL:", public_url)

!streamlit run app.py --server.port 8501 --server.address 0.0.0.0